In [62]:
import numpy as np

N = 5
# 比特数2^N
n_qubits = N**2

# 距离矩阵
D = np.array([[0,  13, 11, 15, 8],
              [13, 0,  7,  14, 9],
              [11, 7,  0,  10, 9],
              [15, 14, 10, 0,  12],
              [8,  9,  9,  12, 0]])

# 约束权重
lambda_val = np.max(D)

# 初始化QUBO矩阵
Q = np.zeros((n_qubits, n_qubits))

# 填充路径代价项
for t in range(N-1):
    for c1 in range(N):
        for c2 in range(N):
            Q[N*t + c1, N*(t+1) + c2] += D[c1, c2]

# 添加约束项
for t in range(N):
    for c1 in range(N):
        for c2 in range(N):
            Q[N*t + c1, N*t + c2] += lambda_val * (-1 if c1 == c2 else 1)

for c in range(N):
    for t1 in range(N):
        for t2 in range(N):
            Q[N*t1 + c, N*t2 + c] += lambda_val * (-1 if t1 == t2 else 1)

In [63]:
import openjij as oj

# 转换为 OpenJij 的 BinaryQuadraticModel
bqm = oj.BinaryQuadraticModel.from_numpy_matrix(
    Q, 
    vartype='BINARY'  # 变量类型：二值 (0或1)
)

print(bqm)

BinaryQuadraticModel({0: -30.0, 1: -30.0, 2: -30.0, 3: -30.0, 4: -30.0, 5: -30.0, 6: -30.0, 7: -30.0, 8: -30.0, 9: -30.0, 10: -30.0, 11: -30.0, 12: -30.0, 13: -30.0, 14: -30.0, 15: -30.0, 16: -30.0, 17: -30.0, 18: -30.0, 19: -30.0, 20: -30.0, 21: -30.0, 22: -30.0, 23: -30.0, 24: -30.0}, {(16, 21): 30.0, (0, 1): 30.0, (1, 8): 14.0, (5, 15): 30.0, (4, 9): 30.0, (0, 2): 30.0, (3, 5): 15.0, (1, 11): 30.0, (4, 14): 30.0, (8, 9): 30.0, (0, 3): 30.0, (3, 4): 30.0, (20, 23): 30.0, (0, 4): 30.0, (5, 8): 30.0, (0, 5): 30.0, (5, 11): 13.0, (4, 5): 8.0, (0, 6): 13.0, (17, 18): 30.0, (5, 13): 15.0, (0, 7): 11.0, (13, 18): 30.0, (0, 8): 15.0, (5, 20): 30.0, (11, 13): 30.0, (3, 7): 10.0, (11, 12): 30.0, (3, 6): 14.0, (8, 23): 30.0, (0, 9): 8.0, (1, 16): 30.0, (0, 10): 30.0, (3, 13): 30.0, (2, 4): 30.0, (11, 14): 30.0, (0, 15): 30.0, (9, 11): 9.0, (0, 20): 30.0, (4, 19): 30.0, (17, 22): 30.0, (1, 2): 30.0, (5, 9): 30.0, (4, 6): 9.0, (17, 23): 10.0, (1, 3): 30.0, (5, 14): 8.0, (21, 23): 30.0, (1, 4): 3

In [64]:
# 创建模拟退火采样器
sampler = oj.SASampler()

# 执行退火（默认参数）
sampleset = sampler.sample(bqm, num_reads=1000)

# 查看结果
print(sampleset)

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 ... 24 energy num_oc.
4    0  0  0  1  0  0  0  1  0  0  0  1  0  0  0  0  0  0 ...  0 -116.0       1
9    1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1 ...  0 -116.0       1
15   0  0  0  1  0  0  0  1  0  0  0  1  0  0  0  0  0  0 ...  0 -116.0       1
16   1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1 ...  0 -116.0       1
18   1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1 ...  0 -116.0       1
38   1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1 ...  0 -116.0       1
40   1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1 ...  0 -116.0       1
54   0  0  0  1  0  0  0  1  0  0  0  1  0  0  0  0  0  0 ...  0 -116.0       1
60   0  0  0  1  0  0  0  1  0  0  0  1  0  0  0  0  0  0 ...  0 -116.0       1
64   0  0  0  1  0  0  0  1  0  0  0  1  0  0  0  0  0  0 ...  0 -116.0       1
66   0  0  0  1  0  0  0  1  0  0  0  1  0  0  0  0  0  0 ...  0 -116.0       1
74   0  0  0  1  0  0  0  1  0  0  0  1 

In [65]:
best_solution = sampleset.first.sample

# 解码函数
def decode_tsp(solution):
    cities = ['A', 'B', 'C', 'D', 'E']
    path = [''] * N
    for idx, val in solution.items():
        if val == 1:
            time_step = idx % N
            city_idx = idx // N
            path[time_step] = cities[city_idx]
    return '->'.join(path) if all(path) else "无效路径"

# 验证并输出
decoded_path = decode_tsp(best_solution)
if len(set(decoded_path.split('->'))) == N:
    print("最优路径:", decoded_path)
else:
    print("未找到合法路径，请检查QUBO约束权重")
    print("当前激活比特:", [k for k, v in best_solution.items() if v == 1])

最优路径: A->C->D->E->B
